In [ ]:
#| hide
from data_system_utilities.file_parsers import yaml
from LTBP.modeling.utils import return_list_of_vars
from IPython.display import Markdown as md

import markdown

import logging, sys
# logging.disable(sys.maxsize)

In [ ]:
#| hide

# This is the code that is needed to power this document for LTBP will need custom changes for your project most likely
from LTBP.data.utils import snowflake_query

experiment_name='BASELINE'
season_year = '2022/23'

sf = snowflake_query()

total_preds = sf.run_sql_str("SELECT COUNT(1) AS TOTAL_POP FROM MachineLearningOutPuts.LTBP.LTBP_PREDICTIONS")

example_output = sf.run_sql_str("SELECT TOP 10 * FROM MachineLearningOutPuts.LTBP.LTBP_PREDICTIONS")

NTILE_breakdown = sf.run_sql_str("""
SELECT 
LikelihoodToBuyPassLabel,
COUNT(1) as Total
FROM (
SELECT ECID
, PROBABILITY
, ntile(5) over (ORDER BY PROBABILITY) as ntile_5
, CASE WHEN ntile_5 = 5 THEN 'High'
  WHEN ntile_5 = 4 THEN 'Med-High'
  WHEN ntile_5 = 3 THEN 'Medium'
  WHEN ntile_5 = 2 THEN 'Low-Med'
  WHEN ntile_5 = 1 THEN 'Low'
  END AS LikelihoodToBuyPassLabel
FROM MACHINELEARNINGOUTPUTS.ltbp.ltbp_PREDICTIONS ltbp
ORDER BY PROBABILITY DESC
)
GROUP BY LikelihoodToBuyPassLabel
""")

example_of_ntile = sf.run_sql_str("""SELECT ECID
, PROBABILITY
, ntile(5) over (ORDER BY PROBABILITY) as ntile_5
, CASE WHEN ntile_5 = 5 THEN 'High'
  WHEN ntile_5 = 4 THEN 'Med-High'
  WHEN ntile_5 = 3 THEN 'Medium'
  WHEN ntile_5 = 2 THEN 'Low-Med'
  WHEN ntile_5 = 1 THEN 'Low'
  END AS LikelihoodToBuyPassLabel
FROM MACHINELEARNINGOUTPUTS.ltbp.ltbp_PREDICTIONS ltbp
ORDER BY PROBABILITY DESC
LIMIT 10
""")

current_season_results = sf.run_sql_str(f"""
SELECT
LikelihoodToBuyPassLabel
, sum(1) Records
, sum(case when ps.ECID is not null
    then 1 else 0 end) FY22_PassHolder
,  Records- FY22_PassHolder as NoPassPurchase
, sum(case when ps.ECID is not null
    then 1 else 0 end)
    / sum(1.0) FY22ConversionRate
FROM (
SELECT ECID, PROBABILITY
, ntile(5) over (ORDER BY PROBABILITY) as ntile_5
, CASE WHEN ntile_5 = 5 THEN 'High'
WHEN ntile_5 = 4 THEN 'Med-High'
WHEN ntile_5 = 3 THEN 'Medium'
WHEN ntile_5 = 2 THEN 'Low-Med'
WHEN ntile_5 = 1 THEN 'Low'
END AS LikelihoodToBuyPassLabel
 FROM MACHINELEARNINGOUTPUTS.LTBP.ltbp_PREDICTIONS ltbp
  ) base
LEFT JOIN BIDE_EDWDB_CUSTOMERMART_PROD.dbo.CustomerPassSummary ps
    on ps.ECID = base.ECID
    and ps.SalesSeason = '{season_year}'
    and ps.Quantity > 0
GROUP BY LikelihoodToBuyPassLabel
ORDER BY max(base.PROBABILITY) desc;
""")

models_dict=yaml.yaml_reader('./LTBP/files/yaml_files/models.yaml')

INFO:data_system_utilities.snowflake.utils:connection to snowflake established...
INFO:data_system_utilities.snowflake.query:executing query
INFO:data_system_utilities.snowflake.query:data loaded from snowflake
INFO:data_system_utilities.snowflake.query:connection to snowflake has been turned off
INFO:data_system_utilities.snowflake.utils:connection to snowflake established...
INFO:data_system_utilities.snowflake.query:executing query
INFO:data_system_utilities.snowflake.query:data loaded from snowflake
INFO:data_system_utilities.snowflake.query:connection to snowflake has been turned off
INFO:data_system_utilities.snowflake.utils:connection to snowflake established...
INFO:data_system_utilities.snowflake.query:executing query
INFO:data_system_utilities.snowflake.query:data loaded from snowflake
INFO:data_system_utilities.snowflake.query:connection to snowflake has been turned off
INFO:data_system_utilities.snowflake.utils:connection to snowflake established...
INFO:data_system_utiliti

# Project Overview

> Likelihood To Buy A Pass (LTBP)

See example sections for how to be able to get your project to a state that will mirror this one

## Install

```sh
pip install LTBP
```

> coming soon private github pypi token to allow for easy install

## LTBP Use Cases

In 2023/24 this project plans on being the replacement to LTR, which historically was used to help subpress media dollar spend for guest segments.

The details of this use case will be filled out as the 22/23 winter season rolls through as we gear up for another year of pass sales.

## LTBP Result Overview

Current Fiscal Season Results:

Development Results: 

- MACHINLEARNINGOUTPUTS.DEV.LTBP_PREDICTIONS

> Note: Development results will be something that we move around as more scale begins to happen, but that isn't solved as of now.

Production Snowflake Table:

- MACHINLEARNINGOUTPUTS.LTBP.LTBP_PREDICTIONS

Total Predictions:

<!-- {{total_preds}} -->

In [ ]:
total_preds

,TOTAL_POP
0,14759852


Example of Prediction Table Output:

<!-- {{example_output}} -->

In [ ]:
example_output

,ECID,PROBABILITY
0,61068721,0.038751
1,130053505,0.492399
2,49940527,0.051959
3,74112965,0.034402
4,32759861,0.004805
5,57941408,0.002742
6,13674145,0.065647
7,60909761,0.009027
8,97172783,0.002126
9,131977740,0.109056


Schema:

|Variable Name| Description|
|:---|:---|
|ECID | Customer Key|
|PROBABILITY| The estimated probability that this ECID will buy a Pass in this Fiscal Year|

Label Logic:

LTBP is composed of five bins low, low-med, medium, med-high high. In the past, there was an attempt to optimize the return rate to be heavy in low, but for simplicity and scalability of this project, DSDE decided to use NTILE, which creates equal buckets for the five labeled groups.

<!-- {{NTILE_breakdown}} -->

In [ ]:
NTILE_breakdown

,LIKELIHOODTOBUYPASSLABEL,TOTAL
0,High,2951970
1,Medium,2951970
2,Med-High,2951970
3,Low-Med,2951971
4,Low,2951971


Example of what the output looks like with ntile(s) assoicated with the raw probability output
<!-- {{example_of_ntile}} -->

In [ ]:
example_of_ntile

,ECID,PROBABILITY,NTILE_5,LIKELIHOODTOBUYPASSLABEL
0,131035762,0.999566,5,High
1,112628819,0.999552,5,High
2,138473097,0.999544,5,High
3,102515974,0.999541,5,High
4,136960602,0.999527,5,High
5,131977201,0.999525,5,High
6,137118754,0.999525,5,High
7,137071085,0.999522,5,High
8,131795763,0.999518,5,High
9,58857511,0.999517,5,High


**Current Season 2022/23 Results**
<!-- **Current Season {{season_year}} Results** -->
<!-- {{current_season_results}} -->

In [ ]:
current_season_results

,LIKELIHOODTOBUYPASSLABEL,RECORDS,FY22_PASSHOLDER,NOPASSPURCHASE,FY22CONVERSIONRATE
0,High,2951970,1108113,1843857,0.375381
1,Med-High,2951970,69263,2882707,0.023463
2,Medium,2951970,30164,2921806,0.010218
3,Low-Med,2951971,24155,2927816,0.008183
4,Low,2951971,12742,2939229,0.004316


## Model Specifics

Description of LTBP Production Model:

<!-- - {{models_dict[experiment_name]['description']}} -->

In [ ]:
models_dict[experiment_name]['description']

'Standard baseline xgb_hyperopt approach status quo of LTBP of the past'

Model Name in side of LTBP Repo:

<!-- - {{models_dict[experiment_name]['model_trainer']}} -->

In [ ]:
models_dict[experiment_name]['model_trainer']

'train_xgb'